# Clean the Consumption and Production Files

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Initialize Spark session
spark = SparkSession.builder \
    .appName("EnergyConsumptionAndProduction") \
    .config("fs.azure.account.key.azddevstorage.dfs.core.windows.net", "YOUR_STORAGE_ACCOUNT_KEY_HERE") \
    .getOrCreate()

In [0]:
# Load a sample Parquet file for one of the locations (e.g., Boston) to inspect the data
consumption_path_sample = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/batch/Chicago/consumption_data/location=Chicago/"
consumption_sample_df = spark.read.format("delta").load(consumption_path_sample)

# Show the schema of the Parquet data to understand the structure
consumption_sample_df.printSchema()

# Show a few sample rows to understand the data
consumption_sample_df.show(5)

root
 |-- date: timestamp (nullable = true)
 |-- location: string (nullable = true)
 |-- residential_consumption: float (nullable = true)
 |-- commercial_consumption: float (nullable = true)
 |-- industrial_consumption: float (nullable = true)

+-------------------+--------+-----------------------+----------------------+----------------------+
|               date|location|residential_consumption|commercial_consumption|industrial_consumption|
+-------------------+--------+-----------------------+----------------------+----------------------+
|2024-12-01 00:00:00| Chicago|                1985.61|               4316.34|               9848.08|
|2024-12-02 00:00:00| Chicago|                 508.75|               2747.64|               7818.95|
|2024-12-03 00:00:00| Chicago|                 925.16|               2922.28|               4226.12|
|2024-12-04 00:00:00| Chicago|                 795.91|               2348.61|               8327.68|
|2024-12-05 00:00:00| Chicago|                 8

## Process Batch Data And Upload the Processed Data to Delta Lake.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from delta import *

# Paths for batch data (consumption and production) and processed data
batch_consumption_path = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/batch/{location}/consumption_data/"
batch_production_path = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/batch/{location}/production_data/"
processed_batch_data_path = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/processed/batch/{location}/"

# Initialize Spark session
# spark = SparkSession.builder.appName("EnergyDataProcessing").getOrCreate()

# Function to process and combine batch consumption and production data
def process_and_combine_batch_data(location):
    try:
        # Load the batch consumption and production data from Delta (without specifying a schema)
        consumption_df = spark.read.format("delta").load(batch_consumption_path.format(location=location))
        production_df = spark.read.format("delta").load(batch_production_path.format(location=location))

        # Perform transformations on consumption data based of the type

       
        consumption_clean_df = consumption_df.withColumnRenamed("date", "timestamp") \
            .withColumn("residential_consumption", F.col("residential_consumption").cast("float")) \
            .withColumn("commercial_consumption", F.col("commercial_consumption").cast("float")) \
            .withColumn("industrial_consumption", F.col("industrial_consumption").cast("float")) \
            .dropna(subset=["timestamp", "residential_consumption", "commercial_consumption", "industrial_consumption"])
        # else:
        #     cleaned_consumption_df = consumption_real_time_df \
        #         .withColumnRenamed("time", "timestamp") \
        #         .withColumn("energy_consumption_kWh", F.col("energy_consumption_kWh").cast("float")) \
        #         .dropna(subset=["timestamp", "energy_consumption_kWh"])  # Drop rows where timestamp or energy_consumption_kWh is null

        # Perform transformations on production data
        production_clean_df = production_df.withColumnRenamed("time", "timestamp") \
            .withColumn("solar_energy", F.col("solar_energy").cast("float")) \
            .withColumn("wind_energy", F.col("wind_energy").cast("float")) \
            .withColumn("hydro_energy", F.col("hydro_energy").cast("float")) \
            .dropna(subset=["timestamp", "solar_energy", "wind_energy", "hydro_energy"])

        # Combine both cleaned DataFrames (consumption and production)
        combined_df = consumption_clean_df.join(production_clean_df, on=["timestamp", "location"], how="outer")

        # Show the final combined DataFrame (before uploading to Delta)
        print(f"Final combined data for {location}:")
        combined_df.show(5, truncate=False)

        # Uncomment below to save the combined data to Delta
        combined_df.write.format("delta").mode("overwrite").partitionBy("location").save(processed_batch_data_path.format(location=location))

        print(f"[INFO] Data for {location} ready for upload (commented out for review).")

    except Exception as e:
        print(f"[ERROR] Error processing data for {location}: {e}")

# Loop through all locations and process data
locations = ["Boston", "New York", "San Francisco", "Chicago"]
for location in locations:
    process_and_combine_batch_data(location)

print("[INFO] Data processing for all locations completed.")


Final combined data for Boston:
+-------------------+--------+-----------------------+----------------------+----------------------+------------+-----------+------------+
|timestamp          |location|residential_consumption|commercial_consumption|industrial_consumption|solar_energy|wind_energy|hydro_energy|
+-------------------+--------+-----------------------+----------------------+----------------------+------------+-----------+------------+
|2024-12-12 00:00:00|Boston  |527.09                 |4662.28               |6496.05               |353.12      |212.93     |1881.35     |
|2024-12-31 00:00:00|Boston  |753.72                 |4697.43               |5167.64               |350.28      |75.29      |671.94      |
|2024-12-20 00:00:00|Boston  |872.45                 |2664.62               |7110.42               |996.17      |82.68      |1629.89     |
|2024-12-13 00:00:00|Boston  |1826.05                |2153.57               |5563.9                |942.53      |135.85     |337.94   

In [0]:
# Load a sample Parquet file for one of the locations (e.g., Boston) to inspect the data for processes
consumption_path_sample = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/real-time/Boston/production_simulation_data/location=Boston"
consumption_sample_df = spark.read.format("delta").load(consumption_path_sample)

# Show the schema of the Parquet data to understand the structure
consumption_sample_df.printSchema()

# Show a few sample rows to understand the data
consumption_sample_df.show(5)

root
 |-- time: timestamp (nullable = true)
 |-- location: string (nullable = true)
 |-- solar_energy: float (nullable = true)
 |-- wind_energy: float (nullable = true)
 |-- hydro_energy: float (nullable = true)

+-------------------+--------+------------+-----------+------------+
|               time|location|solar_energy|wind_energy|hydro_energy|
+-------------------+--------+------------+-----------+------------+
|2025-01-10 15:40:00|  Boston|       34.91|     276.25|         0.0|
|2025-01-10 15:41:00|  Boston|       35.07|     275.42|         0.0|
|2025-01-10 15:42:00|  Boston|       35.23|     274.58|         0.0|
|2025-01-10 15:43:00|  Boston|        35.4|     273.75|         0.0|
|2025-01-10 15:44:00|  Boston|       35.56|     272.92|         0.0|
+-------------------+--------+------------+-----------+------------+
only showing top 5 rows



## Processing Real-Time Data and Uploading the Processed Data to the Delta Lake

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from delta import *

# Paths for batch data (consumption and production) and processed data
real_time_consumption_path = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/real-time/{location}/energy_consumption_data/"
real_time_production_path = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/real-time/{location}/production_simulation_data/"
processed_real_time_data_path = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/processed/real-time/{location}/"

# Initialize Spark session
# spark = SparkSession.builder.appName("EnergyDataProcessing").getOrCreate()

# Function to process and combine batch consumption and production data
def process_and_combine_real_time_data(location):
    try:
        # Load the batch consumption and production data from Delta (without specifying a schema)
        consumption_df = spark.read.format("delta").load(real_time_consumption_path.format(location=location))
        production_df = spark.read.format("delta").load(real_time_production_path.format(location=location))

        # Perform transformations on consumption data based of the type

       
        # consumption_clean_df = consumption_df.withColumnRenamed("date", "timestamp") \
        #     .withColumn("residential_consumption", F.col("residential_consumption").cast("float")) \
        #     .withColumn("commercial_consumption", F.col("commercial_consumption").cast("float")) \
        #     .withColumn("industrial_consumption", F.col("industrial_consumption").cast("float")) \
        #     .dropna(subset=["timestamp", "residential_consumption", "commercial_consumption", "industrial_consumption"])
        # # else:
        consumption_clean_df = consumption_df \
            .withColumnRenamed("time", "timestamp") \
            .withColumn("energy_consumption_kWh", F.col("energy_consumption_kWh").cast("float")) \
            .dropna(subset=["timestamp", "energy_consumption_kWh"])  # Drop rows where timestamp or energy_consumption_kWh is null

        # Perform transformations on production data
        production_clean_df = production_df.withColumnRenamed("time", "timestamp") \
            .withColumn("solar_power", F.col("solar_energy").cast("float")) \
            .withColumn("wind_energy", F.col("wind_energy").cast("float")) \
            .withColumn("hydro_energy", F.col("hydro_energy").cast("float")) \
            .dropna(subset=["timestamp", "solar_energy", "wind_energy", "hydro_energy"])

        # Combine both cleaned DataFrames (consumption and production)
        combined_df = consumption_clean_df.join(production_clean_df, on=["timestamp", "location"], how="outer")

        # Show the final combined DataFrame (before uploading to Delta)
        print(f"Final combined data for {location}:")
        combined_df.show(5, truncate=False)

        # Uncomment below to save the combined data to Delta
        combined_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").partitionBy("location").save(processed_real_time_data_path.format(location=location))

        print(f"[INFO] Data for {location} ready for upload (commented out for review).")

    except Exception as e:
        print(f"[ERROR] Error processing data for {location}: {e}")

# Loop through all locations and process data
locations = ["Boston", "New York", "San Francisco", "Chicago"]
for location in locations:
    process_and_combine_real_time_data(location)

print("[INFO] Data processing for all locations completed.")


Final combined data for Boston:
+--------------------------+--------+--------+---------+-------+----------------------+------------+-----------+------------+-----------+
|timestamp                 |location|latitude|longitude|country|energy_consumption_kWh|solar_energy|wind_energy|hydro_energy|solar_power|
+--------------------------+--------+--------+---------+-------+----------------------+------------+-----------+------------+-----------+
|2025-01-12 07:12:32.752234|Boston  |42.3601 |-71.0589 |USA    |155.3                 |NULL        |NULL       |NULL        |NULL       |
|2025-01-12 04:12:32.752234|Boston  |42.3601 |-71.0589 |USA    |155.3                 |NULL        |NULL       |NULL        |NULL       |
|2025-01-12 03:12:32.752234|Boston  |42.3601 |-71.0589 |USA    |155.3                 |NULL        |NULL       |NULL        |NULL       |
|2025-01-12 12:12:32.752234|Boston  |42.3601 |-71.0589 |USA    |155.3                 |NULL        |NULL       |NULL        |NULL       |
|2

In [0]:
# Load a sample Parquet file for one of the locations (e.g., Boston) to inspect the data for processes
consumption_path_sample = "abfss://<container-name>@<storage-acc>.dfs.core.windows.net/delta/processed/real-time/Boston/location=Boston"
consumption_sample_df = spark.read.format("delta").load(consumption_path_sample)

# Show the schema of the Parquet data to understand the structure
consumption_sample_df.printSchema()

# Show a few sample rows to understand the data
consumption_sample_df.show(5)

root
 |-- timestamp: timestamp (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- country: string (nullable = true)
 |-- energy_consumption_kWh: float (nullable = true)
 |-- solar_energy: float (nullable = true)
 |-- wind_energy: float (nullable = true)
 |-- hydro_energy: float (nullable = true)
 |-- solar_power: float (nullable = true)

+--------------------+--------+--------+---------+-------+----------------------+------------+-----------+------------+-----------+
|           timestamp|location|latitude|longitude|country|energy_consumption_kWh|solar_energy|wind_energy|hydro_energy|solar_power|
+--------------------+--------+--------+---------+-------+----------------------+------------+-----------+------------+-----------+
|2025-01-12 07:12:...|  Boston| 42.3601| -71.0589|    USA|                 155.3|        NULL|       NULL|        NULL|       NULL|
|2025-01-12 04:12:...|  Boston| 42.3601| 